In [19]:
import torch
import argparse
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import homework
from homework import dense_transforms
from homework import utils
import homework.train
from homework import models
from importlib import reload
reload(homework), reload(models), reload(homework.train), reload(homework.utils);

In [20]:
from collections import namedtuple
args = namedtuple('args', 'log_dir other_args')
args.log_dir = 'log'
args.learning_rate = 0.001
args.num_epoch = 20
args.gamma = 0
args.continue_training = False
lo, la = homework.train.train(args)

In [23]:
gamma = 0
DENSE_CLASS_DISTRIBUTION = [0.77357634, 0.11783845, 0.10858521]
#w = torch.as_tensor(DENSE_CLASS_DISTRIBUTION)**(-gamma)
loss = torch.nn.BCEWithLogitsLoss()#weight=w / w.mean())

In [32]:
loss(torch.ones(100)*2, torch.ones(100)*2)

tensor(-1.8731)

In [631]:
la.permute((0,2,3,1))[0].shape

torch.Size([96, 128, 3])

In [637]:
la.permute((0,2,3,1))[0].shape

torch.Size([96, 128, 3])

In [657]:
loss(torch.zeros(1), torch.zeros(1))#la.permute((0,2,3,1))[0])

tensor(0.)

In [ ]:
loss

In [502]:
!python -m grader homework -v

Loading assignment
Loading grader
 * extract_peak                                      
  - return value                                       [ passed ]
  - peak extraction                                    [ passed ]
  - radius=1                                           [ passed ]
  - randomized test                                    [ passed ]
 --------------------------------------------------    [  20 /  20 ]

 * Detector                                          
  - return value                                       [ passed ]
 --------------------------------------------------    [   5 /   5 ]

  - Function not implemented: Detector.detect_with_size
 --------------------------------------------------    [   0 /   9 ]

 * Detection model                                   
  - Average precision (inside box c=0)                 [ AP = 0.129 ]
  - Average precision (inside box c=1)                 [ AP = 0.064 ]
  - Average precision (inside box c=2)                 [ Crash "divi

In [499]:
detections = []
for i in range(3):
    heatmap = image[i]
    peaks = extract_peak(heatmap)
    [detections.append((i, *p)) for p in peaks]

############ shape: torch.Size([50, 50])
7 -5 100
tensor(1.)
tensor(2499)
[(1.0, 0, 0), (1.0, 0, 0), (1.0, 0, 0), (1.0, 0, 0), (1.0, 0, 1), (1.0, 0, 2), (1.0, 0, 3), (1.0, 0, 4), (1.0, 0, 5), (1.0, 0, 6), (1.0, 0, 7), (1.0, 0, 8), (1.0, 0, 9), (1.0, 0, 10), (1.0, 0, 11), (1.0, 0, 12), (1.0, 0, 13), (1.0, 0, 14), (1.0, 0, 15), (1.0, 0, 16)]
############ shape: torch.Size([50, 50])
7 -5 100
tensor(1.)
tensor(2499)
[(1.0, 0, 0), (1.0, 0, 0), (1.0, 0, 0), (1.0, 0, 0), (1.0, 0, 1), (1.0, 0, 2), (1.0, 0, 3), (1.0, 0, 4), (1.0, 0, 5), (1.0, 0, 6), (1.0, 0, 7), (1.0, 0, 8), (1.0, 0, 9), (1.0, 0, 10), (1.0, 0, 11), (1.0, 0, 12), (1.0, 0, 13), (1.0, 0, 14), (1.0, 0, 15), (1.0, 0, 16)]
############ shape: torch.Size([50, 50])
7 -5 100
tensor(1.)
tensor(2499)
[(1.0, 0, 0), (1.0, 0, 0), (1.0, 0, 0), (1.0, 0, 0), (1.0, 0, 1), (1.0, 0, 2), (1.0, 0, 3), (1.0, 0, 4), (1.0, 0, 5), (1.0, 0, 6), (1.0, 0, 7), (1.0, 0, 8), (1.0, 0, 9), (1.0, 0, 10), (1.0, 0, 11), (1.0, 0, 12), (1.0, 0, 13), (1.0, 0, 14), (1